In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["font.family"] = ["Times New Roman"]

# 数据
eta = []
acc = []
pre = []
rec = []
f1 = []
auc = []

plt.figure(figsize=(8, 7), dpi=300)

plt.plot(eta, acc, marker='o', markersize=8, color='#FF4500', linewidth=3.5, label='ACC')
plt.plot(eta, f1, marker='d', markersize=8, color='#1E90FF', linewidth=3.5, label='F1')

plt.plot(eta, pre, marker='s', markersize=6, color='#32CD32', linewidth=2, alpha=0.7, label='PRE')
plt.plot(eta, rec, marker='^', markersize=6, color='#9370DB', linewidth=2, alpha=0.7, label='REC')
plt.plot(eta, auc, marker='*', markersize=10, color='#FFD700', linewidth=2, alpha=0.7, label='AUC')

for x, y in zip(eta, acc):
    plt.annotate(f'{y:.2f}', (x, y), textcoords='offset points',
                 xytext=(0, 8), ha='center', fontsize=10)

for x, y in zip(eta, f1):
    plt.annotate(f'{y:.2f}', (x, y), textcoords='offset points',
                 xytext=(0, -12), ha='center', fontsize=10)

plt.title('The classification task', fontsize=18)
plt.xlabel('$\eta_1$', fontsize=16)
plt.ylabel('Value(%)', fontsize=16)
plt.xticks(eta)
plt.yticks(np.arange(40, 105, 5))  # 更精细的纵坐标刻度
plt.grid(True, linestyle='--', alpha=0.6)

plt.legend(loc='lower right', fontsize=12, framealpha=1)

best_acc_idx = acc.index(max(acc))
best_f1_idx = f1.index(max(f1))
plt.scatter([eta[best_acc_idx]], [max(acc)], color='red', s=100, zorder=5)
plt.scatter([eta[best_f1_idx]], [max(f1)], color='blue', s=100, zorder=5)

plt.tight_layout()
plt.savefig('classification_loss.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = ["Times New Roman"]

# 回归数据数据
rho = []
mse = []
mae = []
r2 = []
pcc = []

fig, ax1 = plt.subplots(figsize=(8, 7), dpi=300)

ax1.plot(rho, mse, marker='o', markersize=8, color='#0066CC', linewidth=3.5, label='MSE')
ax1.plot(rho, mae, marker='s', markersize=6, color='#66B3FF', linewidth=2, alpha=0.7, label='MAE')
ax1.set_ylabel('The values of MSE and MAE', fontsize=14)
ax1.tick_params(axis='y', labelcolor="#0000FF")
ax1.set_ylim(bottom=3, top=8.5)

for x, y in zip(rho, mse):
    plt.annotate(f'{y:.2f}', (x, y), textcoords='offset points',
                 xytext=(0, -12), ha='center', fontsize=10)

ax2 = ax1.twinx()
ax2.plot(rho, r2, marker='^', markersize=6, color='#008000', linewidth=2, alpha=0.7, label='$R^2$')
ax2.plot(rho, pcc, marker='d', markersize=8, color='#80CC80', linewidth=3.5, label='PCC')
ax2.set_ylabel('The values of PCC and $R^2$', fontsize=14)
ax2.tick_params(axis='y', labelcolor="#00FF00")
ax2.set_ylim(bottom=0.35, top=0.9)

for x, y in zip(rho, pcc):
    plt.annotate(f'{y:.2f}', (x, y), textcoords='offset points',
                 xytext=(0, 8), ha='center', fontsize=10)

plt.title('The regression task', fontsize=18)
ax1.set_xlabel('$\\rho_2$', fontsize=16)
plt.xticks(rho, fontsize=14)
plt.grid(True, linestyle='--', alpha=0.4)
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
plt.legend(lines1 + lines2, labels1 + labels2,
           loc='lower right', fontsize=12, framealpha=1)

plt.tight_layout()
plt.savefig('regression_loss.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# ==================== Data input and organization ====================
data = {}

df = pd.DataFrame(data)

# ==================== Calculate the differences and statistics ====================
models = df.columns[1:]
n_models = len(models)
mean_diff = []
ci_lower = []
ci_upper = []
p_values = []

for model in models:
    diff = df["ours"] - df[model]
    t_stat, p_val = stats.ttest_rel(df["ours"], df[model])
    md = diff.mean()
    se = diff.std() / np.sqrt(len(diff))
    ci = stats.t.interval(0.95, len(diff)-1, loc=md, scale=se)
    mean_diff.append(md)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    p_values.append(p_val)

results_df = pd.DataFrame({
    "model": models,
    "mean_diff": mean_diff,
    "ci_lower": ci_lower,
    "ci_upper": ci_upper,
    "p_value": p_values
})

# ==================== Draw a vertical forest map ====================
plt.figure(figsize=(8, 10), dpi=300)
sns.set_style("whitegrid", {"grid.linestyle": "--"})

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16

x_pos = np.arange(n_models)

plt.errorbar(
    x=x_pos,
    y=results_df["mean_diff"],
    yerr=np.vstack((results_df["mean_diff"] - results_df["ci_lower"],  # 下限差
                    results_df["ci_upper"] - results_df["mean_diff"])),  # 上限差
    fmt="o",
    ecolor="#2c7fb8",
    color="#668B8B",
    ms=8,
    capsize=5,
    lw=1.5,
    zorder=3
)

plt.axhline(y=0, color="red", linestyle="--", linewidth=1.2, zorder=0)

for i, row in results_df.iterrows():
    plt.text(x_pos[i], -0.25, row["model"], ha="center", va="top",
             rotation=45, fontweight="bold", fontsize=12)

    sig = "*" * sum([row["p_value"] < 0.001, row["p_value"] < 0.01, row["p_value"] < 0.05])
    plt.text(x_pos[i], row["ci_upper"] + 0.2,
             f"Δ={row['mean_diff']:.2f}% \n(p={row['p_value']:.4f}{sig})",
             ha="center", va="bottom", fontsize=10)

models = df.columns[1:]

plt.ylabel("Performance Difference (ours - Baseline) [Accuracy %]", fontsize=14)
plt.xticks(x_pos, [])
plt.ylim(0, max(results_df["ci_upper"]) + 2)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from lifelines.statistics import logrank_test

# ==================== Data input and organization ====================
data = {}

df = pd.DataFrame(data)

# ==================== Calculate the differences and statistics ====================
models = df.columns[1:]
n_models = len(models)
mean_diff = []
ci_lower = []
ci_upper = []
p_values = []
hr_values = []
hr_lower = []
hr_upper = []

def create_survival_data(our_data, model_data):
    event_times = np.concatenate([our_data, model_data])
    events = np.concatenate([np.ones_like(our_data), np.zeros_like(model_data)])
    groups = np.concatenate([np.ones_like(our_data), np.zeros_like(model_data)])

    survival_df = pd.DataFrame({
        'time': event_times,
        'event': events,
        'group': groups
    })

    return survival_df

for model in models:
    diff = df["ours"] - df[model]
    t_stat, p_val = stats.ttest_rel(df["ours"], df[model])
    md = diff.mean()
    se = diff.std() / np.sqrt(len(diff))
    ci = stats.t.interval(0.95, len(diff)-1, loc=md, scale=se)

    survival_data = create_survival_data(df["ours"], df[model])

    results = logrank_test(
        survival_data[survival_data['group'] == 1]['time'],
        survival_data[survival_data['group'] == 0]['time'],
        event_observed_A=survival_data[survival_data['group'] == 1]['event'],
        event_observed_B=survival_data[survival_data['group'] == 0]['event']
    )

    hr = np.exp(results.test_statistic / (len(df) * 2))

    z = 1.96
    se_log_hr = np.sqrt(1/sum(survival_data['event'] == 1) + 1/sum(survival_data['event'] == 0))
    hr_ci_lower = np.exp(np.log(hr) - z * se_log_hr)
    hr_ci_upper = np.exp(np.log(hr) + z * se_log_hr)

    mean_diff.append(md)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    p_values.append(p_val)
    hr_values.append(hr)
    hr_lower.append(hr_ci_lower)
    hr_upper.append(hr_ci_upper)

results_df = pd.DataFrame({
    "Model": models,
    "Mean Difference (%)": mean_diff,
    "95% CI Lower (%)": ci_lower,
    "95% CI Upper (%)": ci_upper,
    "P-value": p_values,
    "Hazard Ratio": hr_values,
    "95% HR Lower": hr_lower,
    "95% HR Upper": hr_upper
})

results_df.to_excel("model_comparison_results.xlsx", index=False)
print("The result has been successfully output to 'model_comparison_results.xlsx'")

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# ==================== Data input and organization ====================
data = {}

df = pd.DataFrame(data)

# ==================== Calculate the differences and statistics ====================
models = df.columns[1:]
n_models = len(models)
mean_diff = []
ci_lower = []
ci_upper = []
p_values = []

for model in models:
    diff = df["ours"] - df[model]
    t_stat, p_val = stats.ttest_rel(df["ours"], df[model])
    md = diff.mean()
    se = diff.std() / np.sqrt(len(diff))
    ci = stats.t.interval(0.95, len(diff)-1, loc=md, scale=se)
    mean_diff.append(md)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    p_values.append(p_val)

results_df = pd.DataFrame({
    "model": models,
    "mean_diff": mean_diff,
    "ci_lower": ci_lower,
    "ci_upper": ci_upper,
    "p_value": p_values
})

# ==================== Draw a vertical forest map ====================
plt.figure(figsize=(8, 10), dpi=300)
sns.set_style("whitegrid", {"grid.linestyle": "--"})
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16

x_pos = np.arange(n_models)

plt.errorbar(
    x=x_pos,
    y=results_df["mean_diff"],
    yerr=[results_df["mean_diff"] - results_df["ci_lower"],
          results_df["ci_upper"] - results_df["mean_diff"]],
    fmt="*",
    ecolor="#2c7fb8",
    color="#DAA520",
    ms=10,
    capsize=6,
    lw=1.5,
    zorder=3
)

plt.axhline(y=0, color="red", linestyle="--", linewidth=1.2, zorder=0)

for i, row in results_df.iterrows():
    plt.text(x_pos[i], -0.012, row["model"],
             ha="center", va="top",
             rotation=45,
             fontweight="bold",
             fontsize=12)

    sig = "*" * sum([row["p_value"] < 0.001, row["p_value"] < 0.01, row["p_value"] < 0.05])
    plt.text(x_pos[i], row["ci_upper"] + 0.01,
             f"Δ={row['mean_diff']:.2f}\n(p={row['p_value']:.4f}{sig})",
             ha="center", va="bottom",
             fontsize=10)

plt.ylabel("Performance Difference (ours - Baseline) [PCC]", fontsize=14)
plt.xticks(x_pos, [])
plt.ylim(0, max(results_df["ci_upper"]) + 0.1)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from lifelines.statistics import logrank_test

# ==================== 数据输入与整理 ====================
data = {}

df = pd.DataFrame(data)

# ==================== 计算差异与统计量 ====================
models = df.columns[1:]  # 基线模型列表
n_models = len(models)
mean_diff = []          # 均值差
ci_lower = []           # 置信区间下限
ci_upper = []           # 置信区间上限
p_values = []           # p值
hr_values = []          # Hazard Ratio值
hr_lower = []           # HR 95%置信区间下限
hr_upper = []           # HR 95%置信区间上限

# 创建生存分析所需的格式化数据
def create_survival_data(our_data, model_data):
    # 假设每个值代表一个"事件时间"，创建生存分析数据
    # 将数据转换为生存分析所需的格式
    event_times = np.concatenate([our_data, model_data])
    # 创建事件指示符：假设我们的模型总是"更好"，因此事件发生
    # 而基线模型事件未发生（因为我们的模型表现更好）
    events = np.concatenate([np.ones_like(our_data), np.zeros_like(model_data)])
    # 创建组别标签：0为基线模型，1为我们的模型
    groups = np.concatenate([np.ones_like(our_data), np.zeros_like(model_data)])

    # 创建DataFrame
    survival_df = pd.DataFrame({
        'time': event_times,
        'event': events,
        'group': groups
    })

    return survival_df

for model in models:
    # 计算每折差异
    diff = df["ours"] - df[model]
    # 配对t检验
    t_stat, p_val = stats.ttest_rel(df["ours"], df[model])
    # 计算均值差和置信区间
    md = diff.mean()
    se = diff.std() / np.sqrt(len(diff))
    ci = stats.t.interval(0.95, len(diff)-1, loc=md, scale=se)

    # 生存分析 - 计算Hazard Ratio
    survival_data = create_survival_data(df["ours"], df[model])

    # 执行log-rank检验
    results = logrank_test(
        survival_data[survival_data['group'] == 1]['time'],
        survival_data[survival_data['group'] == 0]['time'],
        event_observed_A=survival_data[survival_data['group'] == 1]['event'],
        event_observed_B=survival_data[survival_data['group'] == 0]['event']
    )

    # 计算Hazard Ratio (HR)
    # HR = exp(coef) 其中coef是Cox模型中组别的系数
    # 对于log-rank检验，HR可以近似为风险比
    # 这里使用Mantel-Haenszel方法计算HR
    hr = np.exp(results.test_statistic / (len(df) * 2))  # 简化的HR计算

    # 计算HR的95%置信区间
    # 使用Wald方法的近似
    z = 1.96  # 95%置信区间的z值
    se_log_hr = np.sqrt(1/sum(survival_data['event'] == 1) + 1/sum(survival_data['event'] == 0))
    hr_ci_lower = np.exp(np.log(hr) - z * se_log_hr)
    hr_ci_upper = np.exp(np.log(hr) + z * se_log_hr)

    # 存储结果
    mean_diff.append(md)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    p_values.append(p_val)
    hr_values.append(hr)
    hr_lower.append(hr_ci_lower)
    hr_upper.append(hr_ci_upper)

# 整理为DataFrame
results_df = pd.DataFrame({
    "Model": models,
    "Mean Difference (%)": mean_diff,
    "95% CI Lower (%)": ci_lower,
    "95% CI Upper (%)": ci_upper,
    "P-value": p_values,
    "Hazard Ratio": hr_values,
    "95% HR Lower": hr_lower,
    "95% HR Upper": hr_upper
})

# 输出到Excel文件
results_df.to_excel("model_comparison_results.xlsx", index=False)
print("结果已成功输出到 'model_comparison_results.xlsx' 文件")

In [ ]:
import matplotlib.pyplot as plt

# 设置全局字体为Times New Roman
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False

def count_residues(file_pattern, reference_residues=None):
    residue_counter_high = defaultdict(int)
    residue_counter_low = defaultdict(int)

    for filename in glob.glob(file_pattern):
        try:
            tm_match = re.search(r'Tm-([0-9\.]+)', filename)
            tm_value = float(tm_match.group(1)[:-1])
            with open(filename, 'r') as f:
                reader = csv.reader(f)
                next(reader)  # 跳过标题行
                for row in reader:
                    if len(row) < 2:
                        continue
                    residue = row[1].strip().capitalize()
                    if residue:
                        if tm_value >= 60:
                            residue_counter_high[residue] += 1
                        else:
                            residue_counter_low[residue] += 1
        except Exception as e:
            print(f"处理文件 {filename} 时出错: {str(e)}")

    # 若提供了参考残基列表，则按参考顺序排序，否则按当前任务的高频残基排序
    if reference_residues:
        sorted_residues = reference_residues
    else:
        sorted_residues = sorted(residue_counter_high.keys(),
                               key=lambda x: residue_counter_high[x], reverse=True)

    return (
        {k: residue_counter_high.get(k, 0) for k in sorted_residues},
        {k: residue_counter_low.get(k, 0) for k in sorted_residues}
    )

if __name__ == "__main__":
    # 处理回归任务（作为排序基准）
    reg_file_pattern = 'F:/Downloads/pLDDT/Test_Graph_top_k/*nodes_relations*.csv'
    reg_high, reg_low = count_residues(reg_file_pattern)
    sorted_residues = list(reg_high.keys())  # 以回归任务的高频残基为排序基准

    # 处理分类任务（使用回归任务的排序基准）
    class_file_pattern = 'F:/Downloads/pLDDT/Test_Classification_Graph_top_k/*nodes_relations*.csv'
    class_high, class_low = count_residues(class_file_pattern, reference_residues=sorted_residues)

    # 计算百分比
    def get_percentage(data_dict, total):
        return {k: (v / total * 100) if total != 0 else 0 for k, v in data_dict.items()}

    total_reg_high = sum(reg_high.values())
    total_reg_low = sum(reg_low.values())
    total_class_high = sum(class_high.values())
    total_class_low = sum(class_low.values())

    reg_high_pct = get_percentage(reg_high, total_reg_high)
    reg_low_pct = get_percentage(reg_low, total_reg_low)
    class_high_pct = get_percentage(class_high, total_class_high)
    class_low_pct = get_percentage(class_low, total_class_low)

    # 准备绘图数据（按排序后的残基顺序）
    residues = sorted_residues
    data = [
        [reg_high_pct[r] for r in residues],     # 回归-高温
        [reg_low_pct[r] for r in residues],       # 回归-低温
        [class_high_pct[r] for r in residues],   # 分类-高温
        [class_low_pct[r] for r in residues]    # 分类-低温
    ]

    # 绘图参数
    fig, ax = plt.subplots(figsize=(16, 9), dpi=300)
    bar_width = 0.18  # 单个柱体宽度（4柱总宽度≈0.72，留0.28作为组间距）
    group_space = 0.3  # 残基组之间的间距
    index = range(len(residues))

    # 定义颜色（分类: 红/橙；回归: 蓝/青）
    colors = ['#FF4444', '#FF9933', '#3366CC', '#66CCEE']
    labels = [
        'Regression (Tm≥60°C)',
        'Regression (Tm<60°C)',
        'Classification (Tm≥60°C)',
        'Classification (Tm<60°C)'
    ]

    # 绘制4个柱体
    for i in range(4):
        # 计算每个柱体的x位置（组内偏移）
        x_pos = [j * (bar_width*4 + group_space) + i*bar_width for j in index]
        ax.bar(x_pos, data[i], bar_width, color=colors[i], edgecolor='black', label=labels[i])

    # 图表设置
    ax.set_xlabel('Residue Type', fontsize=16)
    ax.set_ylabel('Percentage (%)', fontsize=16)

    # 设置x轴刻度（对准组中心）
    ax.set_xticks([j*(bar_width*4 + group_space) + bar_width*1.5 for j in index])
    ax.set_xticklabels(residues, rotation=45, ha='center', fontsize=15)

    ax.grid(axis='y', linestyle='--', alpha=0.7, zorder=0)  # 网格置于底层
    ax.legend(loc='upper right', fontsize=16)  # 分两列显示图例

    plt.tight_layout()
    plt.savefig('combined_task_residue_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

    # 统计输出
    print("\n===== 任务统计摘要 =====")
    print("回归任务：")
    print(f"Tm≥60°C残基种类: {len(reg_high)}, Tm<60°C残基种类: {len(reg_low)}")
    print("分类任务：")
    print(f"Tm≥60°C残基种类: {len(class_high)}, Tm<60°C残基种类: {len(class_low)}")
    # 输出所有残基的占比详情（表格形式）
    print("\n===== 所有残基占比详情 =====")
    # 表头
    header = f"{'残基':<8} | {'回归(Tm≥60%)':>12} | {'回归(Tm<60%)':>12} | {'分类(Tm≥60%)':>12} | {'分类(Tm<60%)':>12}"
    print(header)
    print("-" * len(header))  # 分隔线

    # 逐行输出残基数据
    for residue in residues:
        rh = reg_high_pct[residue]
        rl = reg_low_pct[residue]
        ch = class_high_pct[residue]
        cl = class_low_pct[residue]
        # 格式化每行数据（保留2位小数，右对齐）
        row = f"{residue:<8} | {rh:>12.2f}% | {rl:>12.2f}% | {ch:>12.2f}% | {cl:>12.2f}%"
        print(row)

In [ ]:
import csv
import glob
from collections import defaultdict
import re

def clean_residue_name(name):
    """确保残基名称首字母大写，其余字母小写"""
    if not name:
        return name
    return name[0].upper() + name[1:].lower()

def calculate_residue_percentage(residue_counts):
    """计算每个残基的占比"""
    total = sum(residue_counts.values())
    return {res: (count/total)*100 for res, count in residue_counts.items()}


def extract_interaction_scores(file_pattern):
    # 存储残基的相互作用力分数
    interaction_scores = defaultdict(lambda: defaultdict(float))
    interaction_counts = defaultdict(lambda: defaultdict(int))
    residue_totals = defaultdict(float)  # 存储每个残基的总作用力分数
    residue_counts = defaultdict(int)
    interactions = ['VDW', 'PIPISTACK', 'HBOND', 'IONIC', 'SSBOND', 'PICATION']

    # 读取CSV文件并提取残基和相互作用力分数
    for filename in glob.glob(file_pattern):
        try:
            # 提取文件名中的Tm值
            tm_match = re.search(r'Tm-([0-9\.]+)', filename)
            if not tm_match:
                continue

            tm_str = tm_match.group(1).rstrip('.')
            tm_value = float(tm_str)

            with open(filename, 'r') as f:
                reader = csv.reader(f)
                next(reader)  # 跳过标题行
                for row in reader:
                    if len(row) < 2:  # 防止行数据不完整
                        continue
                    residue = row[1].strip().upper()
                    residue_counts[residue] += 1
                    if residue:
                        for count, i in enumerate(interactions):
                            protein = re.search(r'graph_([^_]+)_nodes', filename)
                            path = f"E:/program/GitHub/protein_wang/data/HNet/test2/{i}/{protein.group(1)}.csv"
                            with open(path, 'r') as g:
                                reader_1 = csv.reader(g)
                                for row_1 in reader_1:
                                    if str(int(row_1[0][2:-6])) == row[0] or str(int(row_1[2][2:-6])) == row[0]:
                                        interaction_scores[residue][i] += float(row[count+2])
                                        interaction_counts[residue][i] += 1
                                # print(interaction_counts[residue][i])
                                # print(residue)

            residue_percent = calculate_residue_percentage(residue_counts)
        except Exception as e:
                print(f"处理文件 {filename} 时出错: {str(e)}")

    return interaction_scores, interaction_counts, residue_percent


def calculate_normalized_scores(interaction_scores, interaction_counts):
    # 计算残基特定的作用力比重
    residue_specific_scores = defaultdict(lambda: defaultdict(float))

    for residue in interaction_scores:
        total = interaction_counts[residue]
        if total > 0:
            for interaction in interaction_scores[residue]:
                # 计算该作用力占该残基总作用力的比重
                residue_specific_scores[residue][interaction] = (
                        interaction_scores[residue][interaction] / total * 100
                )  # 转换为百分比

    return residue_specific_scores


def plot_interaction_heatmap(residue_specific_scores):
    if not residue_specific_scores:
        print("没有可用的数据来绘制热图")
        return

    # 创建一个残基列表和相互作用力列表
    residues = sorted(residue_specific_scores.keys())
    interactions = ['VDW', 'PIPISTACK', 'HBOND', 'IONIC', 'SSBOND', 'PICATION']

    # 提取每个残基对应的标准化相互作用力分数
    formatted_residues = [clean_residue_name(res) for res in residues]

    scores_matrix = np.array(
        [[residue_specific_scores[residue][interaction] for interaction in interactions]
         for residue in residues]
    )

    # 使用 seaborn 绘制热图
    plt.figure(figsize=(12, 8), dpi=300)
    sns.heatmap(
        scores_matrix,
        annot=True,
        fmt=".2f",  # 显示一位小数
        cmap="YlGnBu",
        xticklabels=interactions,
        yticklabels=formatted_residues,
        cbar_kws={'label': 'Percentage of Residue Total (%)'}
    )
    plt.xlabel("Interaction Types", fontsize=14)
    plt.ylabel("Residue Types", fontsize=14)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    # 显示热图
    plt.savefig("residue_specific_interaction_heatmap.png", dpi=300)
    plt.show()


def save_to_csv(df, filename="residue_interaction_distribution.csv"):
    # 保存为CSV文件
    df.to_csv(filename, index=False, float_format="%.2f")
    print(f"数据已保存到 {filename}")


def save_to_dataframe(residue_specific_scores):
    # 创建DataFrame
    interactions = ['VDW', 'PIPISTACK', 'HBOND', 'IONIC', 'SSBOND', 'PICATION', 'SELF']
    residues = sorted(residue_specific_scores.keys())

    # 准备数据
    data = []
    for residue in residues:
        row = {'Residue': residue}
        for interaction in interactions:
            row[interaction] = residue_specific_scores[residue].get(interaction, 0)
        data.append(row)

    # 创建DataFrame
    df = pd.DataFrame(data)

    # 添加总计行
    totals = df[interactions].sum()
    totals['Residue'] = 'TOTAL'
    df = df.append(totals, ignore_index=True)

    return df


if __name__ == "__main__":
    file_pattern = "F:/Downloads/pLDDT/Test_Graph_top_k/*nodes_relations*.csv"  # 文件路径
    # file_pattern = "F:/dataset/结果/protein_wang/regression/测试集的关键节点/Top_k/*nodes_relations*.csv"  # 文件路径
    # 提取原始分数
    interaction_scores, interaction_counts, residue_percent = extract_interaction_scores(file_pattern)

    print("\n残基数量占比统计:")
    for res, percent in sorted(residue_percent.items()):
        print(f"{res}: {percent:.2f}%")

    # 计算残基特定的作用力比重
    residue_specific_scores = calculate_normalized_scores(interaction_scores, interaction_counts)

    # 绘制热图
    plot_interaction_heatmap(residue_specific_scores)

    # 转换为DataFrame
    df = save_to_dataframe(residue_specific_scores)

    # 打印表格
    print("\n残基相互作用力分布表(百分比):")
    print(df.to_string(index=False, float_format="%.2f"))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

residue_percent = {
    'His': 0.42, 'Phe': 0.25, 'Trp': 0.98, 'Arg': 13.38, 'Lys': 3.09,
    'Cys': 16.00, 'Gln': 25.99, 'Ala': 5.66, 'Leu': 0.46, 'Val': 0.85
}

interaction_data = {
    'Residue': ['His', 'Phe', 'Trp', 'Arg', 'Lys', 'Cys', 'Gln', 'Ala', 'Leu', 'Val'],
    'VDW': [18.73, 15.23, 18.06, 15.22, 14.96, 15.29, 15.13, 17.99, 15.06, 14.86],
    'PIPISTACK': [1.35, 1.76, 1.24, 0.89, 0.88, 1.05, 1.00, 0.94, 0.52, 1.20],
    'HBOND': [16.55, 15.53, 17.82, 14.64, 13.92, 14.38, 14.49, 15.18, 16.76, 15.02],
    'IONIC': [0.19, 0.62, 0.76, 0.94, 0.78, 0.82, 0.85, 0.87, 0.57, 0.64],
    'SSBOND': [0.04, 0.00, 0.06, 0.02, 0.05, 0.03, 0.03, 0.05, 0.00, 0.00],
    'PICATION': [0.38, 0.56, 0.46, 0.32, 0.25, 0.38, 0.32, 0.32, 0.34, 0.30],
    'SELF': [62.75, 66.30, 61.60, 67.97, 69.16, 68.05, 68.18, 64.64, 66.76, 67.99]
}

# ==============================
# 数据预处理
# ==============================
# 图3：转换为DataFrame并按占比排序
df_node = pd.DataFrame.from_dict(residue_percent, orient='index', columns=['Abundance (%)'])
df_node = df_node.sort_values(by='Abundance (%)', ascending=False)

# 图4：转换为DataFrame并设置索引
df_edge = pd.DataFrame(interaction_data).set_index('Residue')
interactions = ['VDW', 'PIPISTACK', 'HBOND', 'IONIC', 'SSBOND', 'PICATION', 'SELF']
df_edge = df_edge[interactions]

# ==============================
# 绘制复合图表
# ==============================
plt.figure(figsize=(18, 8))

# 左图：图3 - 残基丰度分布（节点角度）
plt.subplot(1, 2, 1)
sns.barplot(
    x=df_node.index,
    y=df_node['Abundance (%)'],
    palette='viridis',
    edgecolor='black'
)
plt.title('(a) Key Residue Abundance in Thermostable Proteins', fontsize=14)
plt.xlabel('Residue Type', fontsize=12)
plt.ylabel('Abundance Percentage (%)', fontsize=12)
plt.xticks(rotation=45, ha='right')
# 添加数值标签
for p in plt.gca().patches:
    height = p.get_height()
    plt.text(p.get_x() + p.get_width()/2., height,
            f'{height:.1f}%',
            ha='center', va='bottom')

# 右图：图4 - 相互作用占比热图（边角度）
plt.subplot(1, 2, 2)
sns.heatmap(
    df_edge,
    annot=True,
    fmt='.1f',
    cmap='YlGnBu',
    cbar_kws={'label': 'Interaction Percentage (%)'}
)
plt.title('(b) Residue-Interaction Strength Distribution', fontsize=14)
plt.xlabel('Interaction Types', fontsize=12)
plt.ylabel('Residue Type', fontsize=12)
plt.xticks(rotation=45, ha='right')

# 添加联动注释（示例：标注关键相互作用）
plt.subplot(1, 2, 2)
plt.text(1, 8, '*PHE的堆积作用显著', fontsize=10, color='red', ha='center')
plt.text(2, 0, '*HIS的氢键主导', fontsize=10, color='red', ha='center')

# 全局调整
plt.tight_layout()
plt.savefig('node_edge_correlation.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import defaultdict

# 设置全局字体为Times New Roman
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams['axes.unicode_minus'] = False

def calculate_normalized_scores():
    # 直接使用您提供的新数据
    data = """
    AA	VDW	PIPISTACK	HBOND	IONIC	SSBOND	PICATION
    ARG	0.2565	0	0.291166667	0.240833333	0	0.2115
    CYS	0.283639706	0	0.31875	0	0.397610294	0
    ALA	0.502484815	0	0.497515185	0	0	0
    SER	0.460498648	0	0.539501352	0	0	0
    GLN	0.462391369	0	0.537608631	0	0	0
    ILE	0.460455362	0	0.539544638	0	0	0
    THR	0.450549451	0	0.549450549	0	0	0
    ASP	0.313587407	0	0.385045568	0.301367026	0	0
    ASN	0.448553534	0	0.551446466	0	0	0
    GLU	0.312371558	0	0.388409371	0.299219071	0	0
    MET	0.460522366	0	0.539477634	0	0	0
    HIS	0.215063319	0.195178849	0.253610309	0.179960009	0	0.156187514
    LYS	0.25061065	0	0.297182869	0.238723335	0	0.213483146
    TRP	0.254317549	0.268802228	0.298746518	0	0	0.178133705
    TYR	0.236837994	0.269020466	0.29151695	0	0	0.20262459
    VAL	0.456045605	0	0.543954395	0	0	0
    GLY	0.460704607	0	0.539295393	0	0	0
    LEU	0.43902439	0	0.56097561	0	0	0
    PHE	0.235758514	0.26377709	0.301702786	0	0	0.19876161
    PRO	0.438692886	0	0.561307114	0	0	0
    """
    # 解析数据
    lines = [line.strip() for line in data.split('\n') if line.strip()]
    header = lines[0].split('\t')
    residues = [line.split('\t')[0] for line in lines[1:]]
    interactions = header[1:]

    residue_specific_scores = defaultdict(lambda: defaultdict(float))
    for line in lines[1:]:
        parts = line.split('\t')
        residue = parts[0]
        for i, interaction in enumerate(interactions):
            score = float(parts[i+1])
            residue_specific_scores[residue][interaction] = score * 100  # 转换为百分比

    return residue_specific_scores, residues, interactions

def plot_interaction_heatmap(residue_specific_scores, residues, interactions):
    if not residue_specific_scores:
        print("无可用数据生成热图。")
        return

    # 构建分数矩阵
    scores_matrix = np.array([
        [residue_specific_scores[residue][interaction] for interaction in interactions]
        for residue in residues
    ])

    # 绘制热图
    plt.figure(figsize=(12, 8), dpi=300)
    sns.heatmap(
        scores_matrix,
        annot=True,
        fmt=".2f",
        cmap="YlGnBu",
        xticklabels=interactions,
        yticklabels=residues,
        cbar_kws={'label': 'Proportion of residue interactions (%)'}
    )
    plt.xlabel("Interaction Types", fontsize=16)
    plt.ylabel("Residue Types", fontsize=16)
    plt.xticks(rotation=45, ha="right", fontsize=12)
    plt.yticks(rotation=0, fontsize=12)
    plt.tight_layout()
    plt.savefig("residue_interaction_heatmap.png", dpi=300)
    plt.show()

if __name__ == "__main__":
    # 计算标准化分数并获取数据结构
    residue_specific_scores, residues, interactions = calculate_normalized_scores()

    # 打印残基相互作用比例
    print("残基-相互作用比例表：")
    for residue in residues:
        print(f"{residue}: {', '.join([f'{interaction}={residue_specific_scores[residue][interaction]:.2f}%' for interaction in interactions])}")

    # 绘制热图
    plot_interaction_heatmap(residue_specific_scores, residues, interactions)
